# MAKING COUNT TABLES
The idea behind this script is producing a new version of the clean, whole dataset in the shape of a count table. Ideally, it all would allow me to run a PCA with the data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
all_data = pd.read_csv("../data/clean_data/ab_data_all_fixed.csv", index_col = 0)
all_data = all_data.loc[~all_data["Assay"].isin(["16S old 1", "16S new 2"])]
all_data.head()

,Assay,antib,sample,Ct,place,type_f,type_g,or_seq,rel_n,rel_ab,log_n,rel_16
0,aacC2,Aminoglycoside,25.0,23.78,ardley,PUR,plastic,arg,9.246982,0.000222,-3.654,41686.938347
1,aacA/aphD,Aminoglycoside,25.0,26.05,ardley,PUR,plastic,arg,1.927525,0.000046,-4.335,41686.938347
2,aac(6')-II,Aminoglycoside,25.0,18.80,ardley,PUR,plastic,arg,288.403150,0.006918,-2.160,41686.938347
3,aphA3,Aminoglycoside,25.0,0.00,ardley,PUR,plastic,arg,NaN,NaN,NaN,41686.938347
4,sat4,Aminoglycoside,25.0,0.00,ardley,PUR,plastic,arg,NaN,NaN,NaN,41686.938347


In [3]:
# Fix some stuff beforehand

# Fix one resistance name that I missed before
all_data.loc[all_data["antib"] == "Phenicol", "antib"] = "Amphenicol"

# Fix a bunch of badly assigned resistances 
all_data.loc[all_data["antib"] == "Other", "or_seq"] = "arg"
all_data.loc[all_data["Assay"] == "dfrA1", "antib"] = "Trimethoprim"
all_data.loc[all_data["Assay"] == "dfrA12", "antib"] = "Trimethoprim"
all_data.loc[all_data["Assay"] == "dfrA1", "or_seq"] = "arg"
all_data.loc[all_data["Assay"] == "dfrA12", "or_seq"] = "arg"
all_data

,Assay,antib,sample,Ct,place,type_f,type_g,or_seq,rel_n,rel_ab,log_n,rel_16
0,aacC2,Aminoglycoside,25.0,23.78,ardley,PUR,plastic,arg,9.246982,0.000222,-3.654,41686.938347
1,aacA/aphD,Aminoglycoside,25.0,26.05,ardley,PUR,plastic,arg,1.927525,0.000046,-4.335,41686.938347
2,aac(6')-II,Aminoglycoside,25.0,18.80,ardley,PUR,plastic,arg,288.403150,0.006918,-2.160,41686.938347
3,aphA3,Aminoglycoside,25.0,0.00,ardley,PUR,plastic,arg,NaN,NaN,NaN,41686.938347
4,sat4,Aminoglycoside,25.0,0.00,ardley,PUR,plastic,arg,NaN,NaN,NaN,41686.938347
...,...,...,...,...,...,...,...,...,...,...,...,...
10309,dfrAB4,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN
10310,dfrC,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN
10311,dfrG,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN
10312,dfrK,Trimethoprim,14.0,0.00,ion,water,control,arg,NaN,NaN,NaN,NaN


In order to obtain datasets as similar as possible to dada2's count tables, I need to make the different ARGs / MGEs (I'm going to separate them) be the rows; each different sample, the columns and each cell, the corresponding NCN.
I'm also going to need to change the sample numbers, as right now there's no correspondance between them and their respective original samples in the global metadata. One step at a time.

## ARG

In [4]:
count_arg = pd.DataFrame()
count_arg.index = all_data.loc[all_data["or_seq"] == "arg"]["Assay"].unique()
count_arg["Assay"] = all_data.loc[all_data["or_seq"] == "arg"]["Assay"].unique() #this is going to serve as a crutch to then just merge stuff onto it
count_arg

,Assay
aacC2,aacC2
aacA/aphD,aacA/aphD
aac(6')-II,aac(6')-II
aphA3,aphA3
sat4,sat4
...,...
dfrAB4,dfrAB4
dfrC,dfrC
dfrG,dfrG
dfrK,dfrK


In [5]:
for sample in all_data["sample"].unique():
    sam_NCN = all_data.loc[(all_data["or_seq"] == "arg") & (all_data["sample"] == sample)][["Assay", "rel_ab"]]
    sam_NCN.rename(columns = {"rel_ab": sample}, inplace = True)
    count_arg = pd.merge(count_arg, sam_NCN, on = "Assay")
count_arg

,Assay,25.0,27.0,20.0,22.0,26.0,23.0,24.0,19.0,21.0,...,12.0,16.0,4.0,5.0,6.0,1.0,2.0,3.0,11.0,14.0
0,aacC2,0.000222,0.000512,0.000056,0.000211,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aacA/aphD,0.000046,0.000039,NaN,0.000149,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aac(6')-II,0.006918,0.006194,NaN,0.006457,0.004699,0.007621,0.008630,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aphA3,NaN,0.002323,NaN,0.000818,0.001205,0.000647,0.000738,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sat4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,dfrAB4,0.000294,NaN,0.000179,0.000208,0.000177,NaN,0.000256,NaN,0.000095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318,dfrC,NaN,NaN,NaN,NaN,NaN,NaN,0.000054,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
319,dfrG,0.000474,0.000471,NaN,NaN,0.000443,NaN,0.000608,0.000277,0.000207,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,dfrK,0.002523,0.002897,0.001633,0.001355,0.002355,0.001374,0.001140,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
count_arg.set_index("Assay", inplace = True) #remove the crutch while fixing row names
count_arg.fillna(value = 0, inplace = True)
count_arg

,25.0,27.0,20.0,22.0,26.0,23.0,24.0,19.0,21.0,17.0,...,12.0,16.0,4.0,5.0,6.0,1.0,2.0,3.0,11.0,14.0
Assay,,,,,,,,,,,,,,,,,,,,,
aacC2,0.000222,0.000512,0.000056,0.000211,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aacA/aphD,0.000046,0.000039,0.000000,0.000149,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aac(6')-II,0.006918,0.006194,0.000000,0.006457,0.004699,0.007621,0.008630,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aphA3,0.000000,0.002323,0.000000,0.000818,0.001205,0.000647,0.000738,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sat4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dfrAB4,0.000294,0.000000,0.000179,0.000208,0.000177,0.000000,0.000256,0.000000,0.000095,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dfrC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000054,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dfrG,0.000474,0.000471,0.000000,0.000000,0.000443,0.000000,0.000608,0.000277,0.000207,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now the second step: changing the sample id's so they match with the global metadata

In [7]:
correspondance_dict = {1: "argl_1", 2: "argl_2", 3: "argl_3", 4: "argl_4", 5: "argl_5",
                       6: "argl_6", 7: "argl_7", 8: "argl_8", 9: "argl_9", 10: "argl_10",
                       11: "argl_11", 12: "argl_12", 13: "argl_13", 14: "argl_14", 15: "argl_15",
                       16: "argl_16", 17: "argl_17", 18: "argl_18", 19: "argl_19", 20: "argl_20",
                       21: "argl_21", 22: "argl_22", 23: "argl_23", 24: "argl_24", 25: "argl_25",
                       26: "argl_26", 27: "argl_27"
                      }
count_arg.rename(columns = correspondance_dict, inplace = True)
count_arg

,argl_25,argl_27,argl_20,argl_22,argl_26,argl_23,argl_24,argl_19,argl_21,argl_17,...,argl_12,argl_16,argl_4,argl_5,argl_6,argl_1,argl_2,argl_3,argl_11,argl_14
Assay,,,,,,,,,,,,,,,,,,,,,
aacC2,0.000222,0.000512,0.000056,0.000211,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aacA/aphD,0.000046,0.000039,0.000000,0.000149,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aac(6')-II,0.006918,0.006194,0.000000,0.006457,0.004699,0.007621,0.008630,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aphA3,0.000000,0.002323,0.000000,0.000818,0.001205,0.000647,0.000738,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sat4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dfrAB4,0.000294,0.000000,0.000179,0.000208,0.000177,0.000000,0.000256,0.000000,0.000095,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dfrC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000054,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dfrG,0.000474,0.000471,0.000000,0.000000,0.000443,0.000000,0.000608,0.000277,0.000207,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


I'm going to make a copy of the count_data, but instead of keeping the NCN as is, I'm taking the log2. Just in case

In [8]:
count_arg_log2 = count_arg + 1
count_arg_log2 = np.log2(count_arg_log2)
count_arg_log2.replace([np.inf, -np.inf], 0, inplace=True) #remember that log(0) = -inf, so we need to make due with those values
count_arg_log2

,argl_25,argl_27,argl_20,argl_22,argl_26,argl_23,argl_24,argl_19,argl_21,argl_17,...,argl_12,argl_16,argl_4,argl_5,argl_6,argl_1,argl_2,argl_3,argl_11,argl_14
Assay,,,,,,,,,,,,,,,,,,,,,
aacC2,0.000320,0.000738,0.000082,0.000305,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aacA/aphD,0.000067,0.000056,0.000000,0.000214,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aac(6')-II,0.009947,0.008909,0.000000,0.009285,0.006763,0.010953,0.012397,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aphA3,0.000000,0.003347,0.000000,0.001180,0.001737,0.000933,0.001064,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sat4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dfrAB4,0.000425,0.000000,0.000258,0.000301,0.000255,0.000000,0.000370,0.000000,0.000137,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dfrC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000078,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dfrG,0.000684,0.000679,0.000000,0.000000,0.000638,0.000000,0.000877,0.000399,0.000299,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## MGE
Same ideas, this time for MGEs

In [9]:
count_mge = pd.DataFrame()
count_mge.index = all_data.loc[all_data["or_seq"] == "mge"]["Assay"].unique()
count_mge["Assay"] = all_data.loc[all_data["or_seq"] == "mge"]["Assay"].unique() #this is going to serve as a crutch to then just merge stuff onto it
#print(count_mge["Assay"].unique())

for sample in all_data["sample"].unique():
    sam_NCN = all_data.loc[(all_data["or_seq"] == "mge") & (all_data["sample"] == sample)][["Assay", "rel_ab"]]
    sam_NCN.rename(columns = {"rel_ab": sample}, inplace = True)
    count_mge = pd.merge(count_mge, sam_NCN, on = "Assay")

count_mge.set_index("Assay", inplace = True) 
count_mge.fillna(value = 0, inplace = True)

count_mge.rename(columns = correspondance_dict, inplace = True)


In [10]:
count_mge_log2 = count_mge + 1
count_mge_log2 = np.log2(count_mge_log2)
count_mge_log2.replace([np.inf, -np.inf], 0, inplace=True) 
count_mge_log2

,argl_25,argl_27,argl_20,argl_22,argl_26,argl_23,argl_24,argl_19,argl_21,argl_17,...,argl_12,argl_16,argl_4,argl_5,argl_6,argl_1,argl_2,argl_3,argl_11,argl_14
Assay,,,,,,,,,,,,,,,,,,,,,
IS1111,1.230117,0.000000,0.000000,1.247345,0.000000,1.434112,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
IS1133,0.000000,0.504665,0.000000,0.438094,0.388498,0.376889,0.370070,0.329351,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
orf37-IS26,1.213040,1.135364,0.000000,0.840506,0.000000,0.894537,0.876254,0.498810,0.000000,0.000000,...,0.000000,0.000000,0.000617,0.000000,0.000000,0.003102,0.000000,0.000000,0.000000,0.000000
orf39-IS26,0.365584,0.490135,0.000000,0.402839,0.000000,0.435491,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001749,0.000000,0.000000,0.000000,0.000000
ISPps1-pseud,0.000000,0.007448,0.000000,0.006015,0.004858,0.012227,0.007979,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000537,0.000000
ISSm2-Xanthob,0.121549,0.114505,0.061779,0.072643,0.000000,0.061779,0.057347,0.073628,0.045841,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000352,0.000000
ISAba3-Acineto,0.000933,0.000933,0.000343,0.000320,0.000643,0.000357,0.000362,0.000425,0.000248,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ISEfm1-Entero,0.000733,0.000684,0.000841,0.000871,0.000748,0.000914,0.001702,0.000748,0.000425,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
TN5,0.005169,0.005576,0.002488,0.010803,0.003279,0.006810,0.006400,0.002896,0.001403,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## FINISHING TOUCHES
Now we just have to save the 4 dataframes as .csv's to bea able to work with them in other environments

In [11]:
count_arg.to_csv("../data/clean_data/count_arg.csv")
count_arg_log2.to_csv("../data/clean_data/count_arg_log2.csv")
count_mge.to_csv("../data/clean_data/count_mge.csv")
count_mge_log2.to_csv("../data/clean_data/count_mge_log2.csv")

Also, a small addendum: I'm going to save the assay-target info in a separate .csv just in case

In [12]:
#Given the way the full dataset is structured, I can get what I want the easiest if I just take the data from one sampling point
assay_data = all_data.loc[all_data["sample"] == 10][["Assay", "antib"]]
assay_data

,Assay,antib
6112,aacC2,Aminoglycoside
6113,aacA/aphD,Aminoglycoside
6114,aac(6')-II,Aminoglycoside
6115,aphA3,Aminoglycoside
6116,sat4,Aminoglycoside
...,...,...
6489,dfrAB4,Trimethoprim
6490,dfrC,Trimethoprim
6491,dfrG,Trimethoprim
6492,dfrK,Trimethoprim


In [13]:
assay_data.to_csv("../data/metadata/assay_metadata.csv")